<a href="https://colab.research.google.com/github/Toshiya1214/ML-for-automation-of-intracochlear-measurements-of-cochlear-implant-electrode-position-/blob/main/Automating_Intracochlear_Measurements_of_Cochlear_Implant_Electrode_Electrode_Position_through_the_Implementation_of_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Threshold Segmentation

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Read the image
image_path = '/Cochlear/0.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply thresholding
_, thresholded = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)

# Display the original and thresholded images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(thresholded, cmap='gray')
plt.title('Thresholded Image')
plt.show()

Canny Edge Detection Segmentation

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Read the image
image_path = '/Cochlear/0.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply Canny edge detection
edges = cv2.Canny(image, threshold1=0, threshold2=30)

# Display the original image and the edges
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(edges, cmap='gray')
plt.title('Edges')
plt.show()


Windowed Level Segmentation

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# Load the original medical image
original_image = cv2.imread('/Cochlear/0.jpg', cv2.IMREAD_GRAYSCALE)

# Set the desired window level (adjust the values as needed)
window_level = -100

# Adjust the image pixel values based on the window level
windowed_image = cv2.convertScaleAbs(original_image, alpha=1.0, beta=window_level)

# Display the images side by side
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_image, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(windowed_image, cmap='gray')
plt.title('Windowed Image (Window Level: {})'.format(window_level))
plt.show()

Gradient Magnitude Segmentation

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# Load the original medical image
original_image = cv2.imread('/Cochlear/0.jpg', cv2.IMREAD_GRAYSCALE)

# Check if the image was successfully loaded
if original_image is None:
    print("Failed to load the image.")
else:
    # Set the desired window level (adjust the values as needed)
    window_level = -100

    # Adjust the image pixel values based on the window level
    windowed_image = cv2.convertScaleAbs(original_image, alpha=1.0, beta=window_level)

    # Apply Gaussian blur to reduce noise (optional)
    blurred_image = cv2.GaussianBlur(windowed_image, (5, 5), 0)

    # Calculate gradients using the Sobel operator
    gradient_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)

    # Calculate the gradient magnitude
    gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)

    # Normalize the gradient magnitude to 0-255
    gradient_magnitude = cv2.normalize(gradient_magnitude, None, 250,270, cv2.NORM_MINMAX, cv2.CV_8U)

    # Display the images side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(windowed_image, cmap='gray')
    plt.title('windowed_image')
    plt.subplot(1, 2, 2)
    plt.imshow(gradient_magnitude, cmap='gray')
    plt.title('Gradient Magnitude')
    plt.show()


Windowed Level Segmentation VS Threshold Segmentation

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# Load the original medical image
original_image = cv2.imread('/Cochlear/0.jpg', cv2.IMREAD_GRAYSCALE)

# Set the desired window level (adjust the values as needed)
window_level = -100

# Adjust the image pixel values based on the window level
windowed_image = cv2.convertScaleAbs(original_image, alpha=1.0, beta=window_level)

# Apply thresholding
_, thresholded = cv2.threshold(windowed_image, 40, 400, cv2.THRESH_BINARY)

# Display the images side by side
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(windowed_image, cmap='gray')
plt.title('Windowed Image (Window Level: {})'.format(window_level))
plt.subplot(1, 2, 2)
plt.imshow(thresholded, cmap='gray')
plt.title('Thresholded Image')
plt.show()

SAM (Segment Anything Model)

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

In [ ]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

In [ ]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
import os

IMAGE_NAME = "/Cochlear/0.jpg"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)

In [ ]:
import cv2
import supervision as sv

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

sam_result = mask_generator.generate(image_rgb)

In [ ]:
print(sam_result[0].keys())

In [ ]:
mask_annotator = sv.MaskAnnotator()

detections = sv.Detections.from_sam(sam_result=sam_result)

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[image_bgr, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

SAM Generate Segmentation with Bounding Box

In [ ]:
mask_predictor = SamPredictor(sam)

In [ ]:
import os

IMAGE_NAME = "/Cochlear/0.jpg"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)

In [ ]:
# helper function that loads an image before adding it to the widget

import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [ ]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image(IMAGE_PATH)
widget

In [ ]:
widget.bboxes

In [ ]:
import numpy as np

# default_box is going to be used if you will not draw any box on image above
default_box = {'x': 68, 'y': 247, 'width': 555, 'height': 678, 'label': ''}

box = widget.bboxes[0] if widget.bboxes else default_box
box = np.array([
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
])

In [ ]:
import cv2
import numpy as np
import supervision as sv

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

mask_predictor.set_image(image_rgb)

masks, scores, logits = mask_predictor.predict(
    box=box,
    multimask_output=True
)

In [ ]:
box_annotator = sv.BoxAnnotator(color=sv.Color.red())
mask_annotator = sv.MaskAnnotator(color=sv.Color.red())

detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
)
detections = detections[detections.area == np.max(detections.area)]

source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[source_image, segmented_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

CNN (Convolutional Neutal Network) model Epochs = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

# Define the paths to the directories containing the data
labeled_image_dir = '/content/drive/MyDrive/CochlearPP42'  # Replace with the actual path
csv_dir = '/content/drive/MyDrive/CochlearCSV42'  # Replace with the actual path

# Preprocessing parameters
image_size = (224, 224)  # Size of the resized images

# Lists to store the preprocessed data
preprocessed_images = []
normalized_coordinates = []

# Loop over each labeled image and corresponding CSV file
for image_index in range(50):
    # Load the labeled image if it exists
    labeled_image_path = os.path.join(labeled_image_dir, f'{image_index:03d}.jpg')
    if not os.path.isfile(labeled_image_path):
        print(f"Labeled image {labeled_image_path} not found. Skipping image {image_index}.")
        continue

    labeled_image = cv2.imread(labeled_image_path)
    labeled_image = cv2.resize(labeled_image, image_size)
    preprocessed_images.append(labeled_image)

    # Load the corresponding CSV file if it exists
    csv_path = os.path.join(csv_dir, f'{image_index:03d}.csv')
    if not os.path.isfile(csv_path):
        print(f"CSV file {csv_path} not found. Skipping image {image_index}.")
        preprocessed_images.pop()  # Remove the corresponding labeled image
        continue

    coordinates_df = pd.read_csv(csv_path)

    # Extract the (x, y) coordinates from the CSV
    coordinates = coordinates_df[['x', 'y']].values

    # Normalize the coordinates
    normalized_coordinates.append(coordinates)  # Add to the list

# Convert the lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
normalized_coordinates = np.array(normalized_coordinates)

# Split the data into training, validation, and test sets
train_images = preprocessed_images[9:]
train_coordinates = normalized_coordinates[9:]

val_images = preprocessed_images[:9]
val_coordinates = normalized_coordinates[:9]


# Proceed with model training using the preprocessed data
import tensorflow as tf
from tensorflow.keras import layers

# Define your CNN model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10*2)  # 20 output units for 10 points (x, y) coordinates
])

# Reshape the target coordinates to match the model's output
train_coordinates_reshaped = train_coordinates.reshape(-1, 10 * 2)
val_coordinates_reshaped = val_coordinates.reshape(-1, 10 * 2)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(train_images, train_coordinates_reshaped, epochs=10, validation_data=(val_images, val_coordinates_reshaped))


In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Plot the training and validation loss
epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'bo-', label='Training Loss')
plt.plot(epochs, validation_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(val_images, val_coordinates_reshaped)
print("Test loss:", test_loss)

from sklearn.metrics import mean_squared_error
# Make predictions on the test set
predictions = model.predict(val_images)
# Reshape the predictions and true coordinates
predictions_reshaped = predictions.reshape(-1, 10 * 2)
# Calculate the mean squared error (MSE)
mse = mean_squared_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Squared Error (MSE):", mse)


from sklearn.metrics import mean_absolute_error
# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import r2_score
# Calculate the R2 score
r2 = r2_score(val_coordinates_reshaped, predictions_reshaped)
print("R2 Score:", r2)

from sklearn.metrics import explained_variance_score
# Calculate the explained variance score
explained_variance = explained_variance_score(val_coordinates_reshaped, predictions_reshaped)
print("Explained Variance Score:", explained_variance)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(val_coordinates_reshaped, predictions_reshaped))
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    epsilon = 1e-10  # small constant to avoid division by zero
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

mape = mean_absolute_percentage_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Absolute Percentage Error (MAPE):", mape)

CNN (Convolutional Neutal Network) model Epochs = 100

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

# Define the paths to the directories containing the data
labeled_image_dir = '/content/drive/MyDrive/CochlearPP42'  # Replace with the actual path
csv_dir = '/content/drive/MyDrive/CochlearCSV42'  # Replace with the actual path

# Preprocessing parameters
image_size = (224, 224)  # Size of the resized images

# Lists to store the preprocessed data
preprocessed_images = []
normalized_coordinates = []

# Loop over each labeled image and corresponding CSV file
for image_index in range(50):
    # Load the labeled image if it exists
    labeled_image_path = os.path.join(labeled_image_dir, f'{image_index:03d}.jpg')
    if not os.path.isfile(labeled_image_path):
        print(f"Labeled image {labeled_image_path} not found. Skipping image {image_index}.")
        continue

    labeled_image = cv2.imread(labeled_image_path)
    labeled_image = cv2.resize(labeled_image, image_size)
    preprocessed_images.append(labeled_image)

    # Load the corresponding CSV file if it exists
    csv_path = os.path.join(csv_dir, f'{image_index:03d}.csv')
    if not os.path.isfile(csv_path):
        print(f"CSV file {csv_path} not found. Skipping image {image_index}.")
        preprocessed_images.pop()  # Remove the corresponding labeled image
        continue

    coordinates_df = pd.read_csv(csv_path)

    # Extract the (x, y) coordinates from the CSV
    coordinates = coordinates_df[['x', 'y']].values

    # Normalize the coordinates
    normalized_coordinates.append(coordinates)  # Add to the list

# Convert the lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
normalized_coordinates = np.array(normalized_coordinates)

# Split the data into training, validation, and test sets
train_images = preprocessed_images[9:]
train_coordinates = normalized_coordinates[9:]

val_images = preprocessed_images[:9]
val_coordinates = normalized_coordinates[:9]


# Proceed with model training using the preprocessed data
import tensorflow as tf
from tensorflow.keras import layers

# Define your CNN model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10*2)  # 20 output units for 10 points (x, y) coordinates
])

# Reshape the target coordinates to match the model's output
train_coordinates_reshaped = train_coordinates.reshape(-1, 10 * 2)
val_coordinates_reshaped = val_coordinates.reshape(-1, 10 * 2)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(train_images, train_coordinates_reshaped, epochs=100, validation_data=(val_images, val_coordinates_reshaped))

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Plot the training and validation loss
epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'bo-', label='Training Loss')
plt.plot(epochs, validation_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(val_images, val_coordinates_reshaped)
print("Test loss:", test_loss)

from sklearn.metrics import mean_squared_error
# Make predictions on the test set
predictions = model.predict(val_images)
# Reshape the predictions and true coordinates
predictions_reshaped = predictions.reshape(-1, 10 * 2)
# Calculate the mean squared error (MSE)
mse = mean_squared_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Squared Error (MSE):", mse)


from sklearn.metrics import mean_absolute_error
# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import r2_score
# Calculate the R2 score
r2 = r2_score(val_coordinates_reshaped, predictions_reshaped)
print("R2 Score:", r2)

from sklearn.metrics import explained_variance_score
# Calculate the explained variance score
explained_variance = explained_variance_score(val_coordinates_reshaped, predictions_reshaped)
print("Explained Variance Score:", explained_variance)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(val_coordinates_reshaped, predictions_reshaped))
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    epsilon = 1e-10  # small constant to avoid division by zero
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

mape = mean_absolute_percentage_error(val_coordinates_reshaped, predictions_reshaped)
print("Mean Absolute Percentage Error (MAPE):", mape)

Export Prediction values

In [ ]:
import pandas as pd

# Define the labels for each point
labels = ['point1', 'point2', 'LW4', 'E4', 'E2', 'LW2', 'LW1', 'E1', 'E3', 'LW3']

# Loop over each image and its corresponding predicted coordinates
for i, coordinates in enumerate(predictions):
    # Reshape the coordinates array
    coordinates = coordinates.reshape(10, 2)

    # Create a DataFrame with the coordinates and labels
    df = pd.DataFrame({'Label': labels, 'x': coordinates[:, 0], 'y': coordinates[:, 1]})

    # Define the output path for the CSV file
    output_path = f'/content/drive/MyDrive/CochlearCNNXY/{i}.csv'

    # Export the DataFrame as a CSV file
    df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
import math
import glob
import os

# Get a list of all CSV files in the directory
csv_files = glob.glob('/content/drive/MyDrive/CochlearCNNXY/*.csv')

# Define a function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Create an empty DataFrame to store the results
results = pd.DataFrame(columns=['SID', 'DL1', 'DL2', 'DL3', 'DL4', 'DE1', 'DE2', 'DE3', 'DE4'])

# Iterate over each CSV file
for file in csv_files:
    # Read the CSV file
    df = pd.read_csv(file)

    # Get the file name without the directory and extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    file_name = f"{file_name}.jpg"  # Add .jpg extension

    # Get the coordinates of Point 2
    point2_row = df[df['Label'] == 'point2']
    point2_x = point2_row['x'].values[0]
    point2_y = point2_row['y'].values[0]

    # Calculate distances between Point 2 and other labeled points
    distances = {}

    label_mapping = {
        'LW1': 'DL1',
        'LW2': 'DL2',
        'LW3': 'DL3',
        'LW4': 'DL4',
        'E1': 'DE1',
        'E2': 'DE2',
        'E3': 'DE3',
        'E4': 'DE4'
    }

    for label, new_label in label_mapping.items():
        row = df[df['Label'] == label]
        x = row['x'].values[0]
        y = row['y'].values[0]
        distance = calculate_distance(point2_x, point2_y, x, y)
        distances[new_label] = distance

    # Create a new DataFrame with the results for the current file
    file_results = pd.DataFrame([distances], index=[file_name])

    # Append the file results to the overall results DataFrame
    results = pd.concat([results, file_results])

# Save the results to a CSV file
results.to_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD.csv', float_format='%.2f')

Calculate Conversion Factors from Preficted values and Actual value\s

In [ ]:
import pandas as pd

# Define the average conversion factors
conversion_factors = {
    'DL1': 0.009903744900565298,
    'DL2': 0.011548086674706151,
    'DL3': 0.011543324708090776,
    'DL4': 0.009351584017787348,
    'DE1': 0.009561528996987883,
    'DE2': 0.011767608791001308,
    'DE3': 0.0117744825198021,
    'DE4': 0.009391603290466594
}

# Read the CSV file with distance values
df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD42.csv')

# Loop over the columns (distance labels)
for column in df.columns[2:]:
    # Get the conversion factor for the current label
    conversion_factor = conversion_factors[column]

    # Convert the distance values to millimeters
    df[column] = df[column] * conversion_factor

# Save the converted distances to a new CSV file
df.to_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD_mm42.csv', index=False, float_format='%.2f')


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Read the CSV file with the actual distance values in millimeters
actual_distances_df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CochlearDMM42.csv')

# Read the CSV file with the newly generated millimeter values
calculated_distances_mm_df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD_mm42.csv')

# Get the columns to evaluate
columns_to_evaluate = ['DL1', 'DL2', 'DL3', 'DL4', 'DE1', 'DE2', 'DE3', 'DE4']

# Calculate the mean absolute error for each column
mae_scores = {}
for column in columns_to_evaluate:
    # Select the first 8 rows from the actual and calculated distances dataframes
    actual_distances = actual_distances_df[column][:8]
    calculated_distances = calculated_distances_mm_df[column][:8]

    # Calculate the mean absolute error
    mae = mean_absolute_error(actual_distances, calculated_distances)
    mae_scores[column] = mae

# Print the mean absolute error for each column
for column, mae in mae_scores.items():
    print(f"Mean Absolute Error for {column}: {mae}")


ResNet-50 (Residual Neural Network 50 layers)  model Epochs=10

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet50

# Define the paths to the directories containing the data
labeled_image_dir = '/content/drive/MyDrive/CochlearPP42'  # Replace with the actual path
csv_dir = '/content/drive/MyDrive/CochlearCSV42'  # Replace with the actual path

# Preprocessing parameters
image_size = (224, 224)  # Size of the resized images

# Lists to store the preprocessed data
preprocessed_images = []
normalized_coordinates = []

# Loop over each labeled image and corresponding CSV file
for image_index in range(50):
    # Load the labeled image
    labeled_image_path = os.path.join(labeled_image_dir, f'{image_index:03d}.jpg')
    if not os.path.isfile(labeled_image_path):
        print(f"Labeled image {labeled_image_path} not found. Skipping image {image_index}.")
        continue

    labeled_image = cv2.imread(labeled_image_path)
    labeled_image = cv2.resize(labeled_image, image_size)
    preprocessed_images.append(labeled_image)

    # Load the corresponding CSV file if it exists
    csv_path = os.path.join(csv_dir, f'{image_index:03d}.csv')
    if not os.path.isfile(csv_path):
        print(f"CSV file {csv_path} not found. Skipping image {image_index}.")
        preprocessed_images.pop()  # Remove the corresponding labeled image
        continue

    coordinates_df = pd.read_csv(csv_path)

    # Extract the (x, y) coordinates from the CSV
    coordinates = coordinates_df[['x', 'y']].values

    # Normalize the coordinates
    normalized_coordinates.append(coordinates)  # Add to the list

# Convert the lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
normalized_coordinates = np.array(normalized_coordinates)

# Split the data into training, validation, and test sets
train_images = preprocessed_images[9:]
train_coordinates = normalized_coordinates[9:]

val_images = preprocessed_images[:9]
val_coordinates = normalized_coordinates[:9]

import torch
import torch.nn as nn
from torchvision.models import resnet50

# Load the pre-trained ResNet-50 model
model = resnet50(pretrained=True)

# Modify the model's last layer for regression
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10 * 2)  # 10 points (x, y) coordinates per sample

# Define the loss function
loss_fn = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
train_images_tensor = torch.from_numpy(train_images).permute(0, 3, 1, 2).float()
train_coordinates_tensor = torch.from_numpy(train_coordinates).float()

val_images_tensor = torch.from_numpy(val_images).permute(0, 3, 1, 2).float()
val_coordinates_tensor = torch.from_numpy(val_coordinates).float()

# Reshape the target tensors
train_coordinates_tensor = train_coordinates_tensor.view(-1, 10 * 2)
val_coordinates_tensor = val_coordinates_tensor.view(-1, 10 * 2)

# Train the model
num_epochs = 10
batch_size = 8

import matplotlib.pyplot as plt

# Lists to store the training and validation loss
train_loss_history = []
val_loss_history = []

# Train the model
for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(train_images), batch_size):
        # Prepare batch
        batch_images = train_images_tensor[i:i + batch_size]
        batch_coordinates = train_coordinates_tensor[i:i + batch_size]

        # Forward pass and optimization
        optimizer.zero_grad()
        outputs = model(batch_images)
        loss = loss_fn(outputs, batch_coordinates)
        loss.backward()
        optimizer.step()

    # Evaluation on the validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(val_images_tensor)
        val_loss = loss_fn(val_outputs, val_coordinates_tensor)

    # Append the training and validation loss to the history
    train_loss_history.append(loss.item())
    val_loss_history.append(val_loss.item())

    # Print the progress
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    if epoch == num_epochs - 1:
        # Plot the training and validation loss for the last epoch
        epochs = range(1, epoch + 2)
        plt.plot(epochs, train_loss_history, 'bo-', label='Training Loss')
        plt.plot(epochs, val_loss_history, 'ro-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()


In [ ]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Set the model to evaluation mode
model.eval()

# Calculate predictions for the validation set
val_predictions = model(val_images_tensor)

# Detach the tensors from the computation graph and convert to NumPy arrays
val_coordinates_numpy = val_coordinates_tensor.detach().numpy()
val_predictions_numpy = val_predictions.detach().numpy()

# Calculate evaluation metrics
val_loss = loss_fn(val_predictions, val_coordinates_tensor)
mse_val = mean_squared_error(val_coordinates_numpy, val_predictions_numpy)
mae_val = mean_absolute_error(val_coordinates_numpy, val_predictions_numpy)
r2_val = r2_score(val_coordinates_numpy, val_predictions_numpy)
explained_variance_val = explained_variance_score(val_coordinates_numpy, val_predictions_numpy)
rmse_val = math.sqrt(mse_val)

# Calculate MAPE if there are no zero values in val_coordinates_numpy
if np.any(val_coordinates_numpy == 0):
    mape_val = np.inf
else:
    mape_val = np.mean(np.abs((val_coordinates_numpy - val_predictions_numpy) / val_coordinates_numpy)) * 100

# Print the evaluation metrics
print(f"Validation Loss: {val_loss:.4f}")
print(f"MSE: {mse_val:.4f}")
print(f"MAE: {mae_val:.4f}")
print(f"R2: {r2_val:.4f}")
print(f"Explained Variance Score: {explained_variance_val:.4f}")
print(f"RMSE: {rmse_val:.4f}")
print(f"MAPE: {mape_val:.4f}%")

ResNet-50 (Residual Neural Network 50 layers)  model Epochs=100

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet50

# Define the paths to the directories containing the data
labeled_image_dir = '/content/drive/MyDrive/CochlearPP42'  # Replace with the actual path
csv_dir = '/content/drive/MyDrive/CochlearCSV42'  # Replace with the actual path

# Preprocessing parameters
image_size = (224, 224)  # Size of the resized images

# Lists to store the preprocessed data
preprocessed_images = []
normalized_coordinates = []

# Loop over each labeled image and corresponding CSV file
for image_index in range(50):
    # Load the labeled image
    labeled_image_path = os.path.join(labeled_image_dir, f'{image_index:03d}.jpg')
    if not os.path.isfile(labeled_image_path):
        print(f"Labeled image {labeled_image_path} not found. Skipping image {image_index}.")
        continue

    labeled_image = cv2.imread(labeled_image_path)
    labeled_image = cv2.resize(labeled_image, image_size)
    preprocessed_images.append(labeled_image)

    # Load the corresponding CSV file if it exists
    csv_path = os.path.join(csv_dir, f'{image_index:03d}.csv')
    if not os.path.isfile(csv_path):
        print(f"CSV file {csv_path} not found. Skipping image {image_index}.")
        preprocessed_images.pop()  # Remove the corresponding labeled image
        continue

    coordinates_df = pd.read_csv(csv_path)

    # Extract the (x, y) coordinates from the CSV
    coordinates = coordinates_df[['x', 'y']].values

    # Normalize the coordinates
    normalized_coordinates.append(coordinates)  # Add to the list

# Convert the lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
normalized_coordinates = np.array(normalized_coordinates)

# Split the data into training, validation, and test sets
train_images = preprocessed_images[9:]
train_coordinates = normalized_coordinates[9:]

val_images = preprocessed_images[:9]
val_coordinates = normalized_coordinates[:9]

import torch
import torch.nn as nn
from torchvision.models import resnet50

# Load the pre-trained ResNet-50 model
model = resnet50(pretrained=True)

# Modify the model's last layer for regression
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10 * 2)  # 10 points (x, y) coordinates per sample

# Define the loss function
loss_fn = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
train_images_tensor = torch.from_numpy(train_images).permute(0, 3, 1, 2).float()
train_coordinates_tensor = torch.from_numpy(train_coordinates).float()

val_images_tensor = torch.from_numpy(val_images).permute(0, 3, 1, 2).float()
val_coordinates_tensor = torch.from_numpy(val_coordinates).float()

# Reshape the target tensors
train_coordinates_tensor = train_coordinates_tensor.view(-1, 10 * 2)
val_coordinates_tensor = val_coordinates_tensor.view(-1, 10 * 2)

# Train the model
num_epochs = 100
batch_size = 8

import matplotlib.pyplot as plt

# Lists to store the training and validation loss
train_loss_history = []
val_loss_history = []

# Train the model
for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(train_images), batch_size):
        # Prepare batch
        batch_images = train_images_tensor[i:i + batch_size]
        batch_coordinates = train_coordinates_tensor[i:i + batch_size]

        # Forward pass and optimization
        optimizer.zero_grad()
        outputs = model(batch_images)
        loss = loss_fn(outputs, batch_coordinates)
        loss.backward()
        optimizer.step()

    # Evaluation on the validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(val_images_tensor)
        val_loss = loss_fn(val_outputs, val_coordinates_tensor)

    # Append the training and validation loss to the history
    train_loss_history.append(loss.item())
    val_loss_history.append(val_loss.item())

    # Print the progress
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    if epoch == num_epochs - 1:
        # Plot the training and validation loss for the last epoch
        epochs = range(1, epoch + 2)
        plt.plot(epochs, train_loss_history, 'bo-', label='Training Loss')
        plt.plot(epochs, val_loss_history, 'ro-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [ ]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Set the model to evaluation mode
model.eval()

# Calculate predictions for the validation set
val_predictions = model(val_images_tensor)

# Detach the tensors from the computation graph and convert to NumPy arrays
val_coordinates_numpy = val_coordinates_tensor.detach().numpy()
val_predictions_numpy = val_predictions.detach().numpy()

# Calculate evaluation metrics
val_loss = loss_fn(val_predictions, val_coordinates_tensor)
mse_val = mean_squared_error(val_coordinates_numpy, val_predictions_numpy)
mae_val = mean_absolute_error(val_coordinates_numpy, val_predictions_numpy)
r2_val = r2_score(val_coordinates_numpy, val_predictions_numpy)
explained_variance_val = explained_variance_score(val_coordinates_numpy, val_predictions_numpy)
rmse_val = math.sqrt(mse_val)

# Calculate MAPE if there are no zero values in val_coordinates_numpy
if np.any(val_coordinates_numpy == 0):
    mape_val = np.inf
else:
    mape_val = np.mean(np.abs((val_coordinates_numpy - val_predictions_numpy) / val_coordinates_numpy)) * 100

# Print the evaluation metrics
print(f"Validation Loss: {val_loss:.4f}")
print(f"MSE: {mse_val:.4f}")
print(f"MAE: {mae_val:.4f}")
print(f"R2: {r2_val:.4f}")
print(f"Explained Variance Score: {explained_variance_val:.4f}")
print(f"RMSE: {rmse_val:.4f}")
print(f"MAPE: {mape_val:.4f}%")

Expoert Prediction values

In [ ]:
import pandas as pd

# Define the labels for each point
labels = ['point1', 'point2', 'LW4', 'E4', 'E2', 'LW2', 'LW1', 'E1', 'E3', 'LW3']

# Loop over each image and its corresponding predicted coordinates
for i, coordinates in enumerate(predicted_coordinates):
    # Create a DataFrame with the coordinates and labels
    df = pd.DataFrame({'Label': labels, 'x': coordinates[:, 0], 'y': coordinates[:, 1]})

    # Define the output path for the CSV file
    output_path = f'/content/drive/MyDrive/CochlearResNetXY/{i}.csv'

    # Export the DataFrame as a CSV file
    df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
import math
import glob
import os

# Get a list of all CSV files in the directory
csv_files = glob.glob('/content/drive/MyDrive/CochlearResNetXY/*.csv')

# Define a function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Create an empty DataFrame to store the results
results = pd.DataFrame(columns=['SID', 'DL1', 'DL2', 'DL3', 'DL4', 'DE1', 'DE2', 'DE3', 'DE4'])

# Iterate over each CSV file
for file in csv_files:
    # Read the CSV file
    df = pd.read_csv(file)

    # Get the file name without the directory and extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    file_name = f"{file_name}.jpg"  # Add .jpg extension

    # Get the coordinates of Point 2
    point2_row = df[df['Label'] == 'point2']
    point2_x = point2_row['x'].values[0]
    point2_y = point2_row['y'].values[0]

    # Calculate distances between Point 2 and other labeled points
    distances = {}

    label_mapping = {
        'LW1': 'DL1',
        'LW2': 'DL2',
        'LW3': 'DL3',
        'LW4': 'DL4',
        'E1': 'DE1',
        'E2': 'DE2',
        'E3': 'DE3',
        'E4': 'DE4'
    }

    for label, new_label in label_mapping.items():
        row = df[df['Label'] == label]
        x = row['x'].values[0]
        y = row['y'].values[0]
        distance = calculate_distance(point2_x, point2_y, x, y)
        distances[new_label] = distance

    # Create a new DataFrame with the results for the current file
    file_results = pd.DataFrame([distances], index=[file_name])

    # Append the file results to the overall results DataFrame
    results = pd.concat([results, file_results])

# Save the results to a CSV file
results.to_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD.csv', float_format='%.2f')

Calculate Conversion Factors from Predicted values and Actual values

In [ ]:
import pandas as pd

# Define the average conversion factors
conversion_factors = {
    'DL1': 0.009903744900565298,
    'DL2': 0.011548086674706151,
    'DL3': 0.011543324708090776,
    'DL4': 0.009351584017787348,
    'DE1': 0.009561528996987883,
    'DE2': 0.011767608791001308,
    'DE3': 0.0117744825198021,
    'DE4': 0.009391603290466594
}

# Read the CSV file with distance values
df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD42.csv')

# Loop over the columns (distance labels)
for column in df.columns[2:]:
    # Get the conversion factor for the current label
    conversion_factor = conversion_factors[column]

    # Convert the distance values to millimeters
    df[column] = df[column] * conversion_factor

# Save the converted distances to a new CSV file
df.to_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD_mmX42.csv', index=False, float_format='%.2f')

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Read the CSV file with the actual distance values in millimeters
actual_distances_df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CochlearDMM42.csv')

# Read the CSV file with the newly generated millimeter values
calculated_distances_mm_df = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD_mmX42.csv')

# Get the columns to evaluate
columns_to_evaluate = ['DL1', 'DL2', 'DL3', 'DL4', 'DE1', 'DE2', 'DE3', 'DE4']

# Calculate the mean absolute error for each column
mae_scores = {}
for column in columns_to_evaluate:
    # Select the first 8 rows from the actual and calculated distances dataframes
    actual_distances = actual_distances_df[column][:8]
    calculated_distances = calculated_distances_mm_df[column][:8]

    # Calculate the mean absolute error
    mae = mean_absolute_error(actual_distances, calculated_distances)
    mae_scores[column] = mae

# Print the mean absolute error for each column
for column, mae in mae_scores.items():
    print(f"Mean Absolute Error for {column}: {mae}")

Calculating Mean Absolute Differences of CNN Prediction model

In [ ]:
import pandas as pd
import numpy as np

# Load the CNN predicted values dataset from a CSV file
calculated_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD_mm42.csv')

# Load the actual values dataset from a CSV file
actual_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CochlearDMM0_8.csv')

# Extract the values from the calculated_data DataFrame
calculated_values = calculated_data.iloc[:, 1:].values

# Extract the values from the actual_data DataFrame
actual_values = actual_data.iloc[:, 1:].values

# Calculate the absolute differences for each column
absolute_diff = np.abs(calculated_values - actual_values)

# Calculate the Mean Absolute Difference (MAD) for each column
mad_per_column = np.mean(absolute_diff, axis=0)

# Calculate the total Mean Absolute Difference (MAD) for all columns combined
mad_total = np.mean(absolute_diff)

# Display the MAD for each column
columns = calculated_data.columns[1:]  # Exclude the first column (image names or identifiers)
for column, mad_value in zip(columns, mad_per_column):
    print("Mean Absolute Difference ({}): {:.6f}".format(column, mad_value))

# Display the total MAD for all columns combined
print("Total Mean Absolute Difference (All Columns): {:.6f}".format(mad_total))


Calculating Mean Absolute Differences of ResNet Prediction model

In [ ]:
import pandas as pd
import numpy as np

# Load the ResNet predicted values dataset from a CSV file
calculated_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD_mmX42.csv')

# Load the actual values dataset from a CSV file
actual_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CochlearDMM0_8.csv')

# Extract the values from the calculated_data DataFrame
calculated_values = calculated_data.iloc[:, 1:].values

# Extract the values from the actual_data DataFrame
actual_values = actual_data.iloc[:, 1:].values

# Calculate the absolute differences for each column
absolute_diff = np.abs(calculated_values - actual_values)

# Calculate the Mean Absolute Difference (MAD) for each column
mad_per_column = np.mean(absolute_diff, axis=0)

# Calculate the total Mean Absolute Difference (MAD) for all columns combined
mad_total = np.mean(absolute_diff)

# Display the MAD for each column
columns = calculated_data.columns[1:]  # Exclude the first column (image names or identifiers)
for column, mad_value in zip(columns, mad_per_column):
    print("Mean Absolute Difference ({}): {:.6f}".format(column, mad_value))

# Display the total MAD for all columns combined
print("Total Mean Absolute Difference (All Columns): {:.6f}".format(mad_total))


Calculating MAD of CNN and ResNet models and Export results in CSV file

In [ ]:
import pandas as pd
import numpy as np

# Load the CNN predicted values dataset from a CSV file
cnn_calculated_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CNNPreD_mm42.csv')

# Load the ResNet predicted values dataset from a CSV file
resnet_calculated_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/ResNetPreD_mmX42.csv')

# Load the actual values dataset from a CSV file
actual_data = pd.read_csv('/content/drive/MyDrive/CochlearDistance42/CochlearDMM0_8.csv')

# Extract the values from the calculated_data DataFrames
cnn_calculated_values = cnn_calculated_data.iloc[:, 1:].values
resnet_calculated_values = resnet_calculated_data.iloc[:, 1:].values

# Extract the values from the actual_data DataFrame
actual_values = actual_data.iloc[:, 1:].values

# Calculate the absolute differences for each model
cnn_absolute_diff = np.abs(cnn_calculated_values - actual_values)
resnet_absolute_diff = np.abs(resnet_calculated_values - actual_values)

# Calculate the Mean Absolute Difference (MAD) for each model
cnn_mad_per_column = np.mean(cnn_absolute_diff, axis=0)
resnet_mad_per_column = np.mean(resnet_absolute_diff, axis=0)

# Combine the MAD results into a DataFrame
mad_results = pd.DataFrame({
    'Model': ['CNN', 'ResNet'],
    'DL1': [cnn_mad_per_column[0], resnet_mad_per_column[0]],
    'DL2': [cnn_mad_per_column[1], resnet_mad_per_column[1]],
    'DL3': [cnn_mad_per_column[2], resnet_mad_per_column[2]],
    'DL4': [cnn_mad_per_column[3], resnet_mad_per_column[3]],
    'DE1': [cnn_mad_per_column[4], resnet_mad_per_column[4]],
    'DE2': [cnn_mad_per_column[5], resnet_mad_per_column[5]],
    'DE3': [cnn_mad_per_column[6], resnet_mad_per_column[6]],
    'DE4': [cnn_mad_per_column[7], resnet_mad_per_column[7]],
    'Total MAD': [np.mean(cnn_absolute_diff), np.mean(resnet_absolute_diff)]
})

# Export the MAD results to a CSV file
mad_results.to_csv('/content/drive/MyDrive/CochlearDistance42/MAD_results.csv', index=False)
